# Laden omgeving en software

In [ ]:
# !git clone https://github.com/dsa-playground/tvt2023.git
# %cd /content/tvt2023/
# !git pull
# !pip install -r requirements.txt -t "tvt2023"
# !pip install pyaml-env

In [1]:
# Imports
import warnings
import pandas as pd
from scripts.preprocess.preprocess_frontend import laden_data, vereenvoudigen_data, aanpassen_data, voeg_passagiers_toe
from scripts.EDA.eda import basis_feiten, visualisatie_ticketklasse, visualisatie_opstapplaats, visualisatie_leeftijd_geslacht, visualisatie_familieleden
from scripts.modeling.modeling_frontend import experimenteer_met_aantal_buren, verdieping_specifiek_model, voorspelling_genereren
from scripts.evaluation.evaluation import geef_belangrijkste_variabelen 

# Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)

# Disable warnings
warnings.filterwarnings('ignore')

# Inleiding

### Use case 'Titanic'
![Laatste foto van de Titanic](https://raw.githubusercontent.com/dsa-playground/tvt2023/main/images/lastphoto_titanic.png)

Plenair is CRISP-DM toegelicht, inclusief wat de acties zijn in de eerste 3 stappen:
1. Business Understanding 
2. Data Understanding
3. Data Preparation

Nu gaan jullie aan de slag! In de cellen hieronder kunnen jullie inzicht krijgen in de data. Kunnen jullie met deze informatie antwoord geven op de bijbehorende vragen? 

### 1. Business Understanding
De hypothese voor deze use case luidt:

*Kan een algoritme voorspellen of een passagier de Titanic heeft overleefd?*

En... *of we het zelf overleefd zouden hebben?*

### 2. Data Understanding
Bekijk in de cel hieronder de originele dataset:

In [2]:
df_train, df_test = laden_data()
display(df_train.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


Zoals je ziet is de dataset in het Engels en soms wat cryptisch weergegeven. Laten we dit vereenvoudigen. 

In [3]:
df_train_clean, df_test_clean = vereenvoudigen_data(df_train, df_test)
display(df_train_clean.head())

,Passagier_Id,Naam,Geslacht,Leeftijd,Aantal_kinderen,Aantal_overige_familieleden,Totaal_aantal_familieleden,Opstapplaats,Ticket_nummer,Ticket_klasse,Ticket_prijs,Cabine_nummer,Overleefd
0,1,"Braund, Mr. Owen Harris",Man,22.000,0,1,1,Engeland,A/5 21171,3e klas,7.250,NaN,Nee
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",Vrouw,38.000,0,1,1,Frankrijk,PC 17599,1e klas,71.283,C85,Ja
2,3,"Heikkinen, Miss. Laina",Vrouw,26.000,0,0,0,Engeland,STON/O2. 3101282,3e klas,7.925,NaN,Ja
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Vrouw,35.000,0,1,1,Engeland,113803,1e klas,53.100,C123,Ja
4,5,"Allen, Mr. William Henry",Man,35.000,0,0,0,Engeland,373450,3e klas,8.050,NaN,Nee


**Opgave 2.1.** Welke variabelen denken jullie dat belangrijk zijn?

**Opgave 2.2.** Welke variabele is het doel?

We kunnen naast de tabel ook een aantal (beknopte) verkennende analyses uitvoeren.  
Druk op de onderstaande play knoppen om dit te doen.

In [4]:
basis_feiten(df=df_train_clean)

Er zitten 891 passagiers in de dataset, daarvan heeft 38.38% het overleefd.
De gemiddelde leeftijd van de passagiers is 30.
 


,Missende waarden (%)
Cabine_nummer,77.104
Leeftijd,19.865
Opstapplaats,0.224
Passagier_Id,0.000
Naam,0.000
Geslacht,0.000
Aantal_kinderen,0.000
Aantal_overige_familieleden,0.000
Totaal_aantal_familieleden,0.000
Ticket_nummer,0.000


**Opgave 2.3.** Welke kolommen hebben missende waarden?

In [ ]:
visualisatie_ticketklasse(df=df_train_clean)

**Opgave 2.4.** In welke klasse hebben (in verhouding) de minste passagiers het overleefd?

In [ ]:
visualisatie_leeftijd_geslacht(df=df_train_clean)

De 3D-grafiek is op verschillende manieren te bekijken:
- Door te scrollen kun je in- en/of uitzoomen
- Rechtermuisknop vasthouden en bewegen leidt tot het draaien van de grafiek
- Je ziet extra informatie als je met de muis over een punt beweegt. 

**Opgave 2.5.** Wat valt je op aan deze (3D) grafiek?

In [5]:
visualisatie_familieleden(df=df_train_clean)

Het percentage overleefde/overleden passagiers ten opzichte van het aantal familieleden.


**Opgave 2.6.** Kijk naar het plaatje op de handout en omcirkel a, b of c. Welke distributie past het beste bij het percentage aantal familieleden dat overleefd heeft?

### 3. Data preparation
Laten we de dataset aanpassen zodat deze geschikt is voor een algoritme. In de cel hieronder kun je de oude én opgeschoonde dataset bekijken.

In [8]:
df_train_num, df_test_num = aanpassen_data(df_train_clean, df_test_clean)
print("Oude dataset:") 
display(df_train_clean.head())
print("Opgeschoonde dataset:")
display( df_train_num.head())

Oude dataset:


,Passagier_Id,Naam,Geslacht,Leeftijd,Aantal_kinderen,Aantal_overige_familieleden,Totaal_aantal_familieleden,Opstapplaats,Ticket_nummer,Ticket_klasse,Ticket_prijs,Cabine_nummer,Overleefd
0,1,"Braund, Mr. Owen Harris",Man,22.000,0,1,1,Engeland,A/5 21171,3e klas,7.250,NaN,Nee
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",Vrouw,38.000,0,1,1,Frankrijk,PC 17599,1e klas,71.283,C85,Ja
2,3,"Heikkinen, Miss. Laina",Vrouw,26.000,0,0,0,Engeland,STON/O2. 3101282,3e klas,7.925,NaN,Ja
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Vrouw,35.000,0,1,1,Engeland,113803,1e klas,53.100,C123,Ja
4,5,"Allen, Mr. William Henry",Man,35.000,0,0,0,Engeland,373450,3e klas,8.050,NaN,Nee


Opgeschoonde dataset:


,,Geslacht,Leeftijd,Aantal_kinderen,Aantal_overige_familieleden,Totaal_aantal_familieleden,Opstapplaats,Ticket_klasse,Ticket_prijs,Overleefd
Passagier_Id,Naam,,,,,,,,,
1,"Braund, Mr. Owen Harris",0,22,0,1,1,0,3,7.250,0
2,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",1,38,0,1,1,1,1,71.283,1
3,"Heikkinen, Miss. Laina",1,26,0,0,0,0,3,7.925,1
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,0,1,1,0,1,53.100,1
5,"Allen, Mr. William Henry",0,35,0,0,0,0,3,8.050,0


**Opgave 3.1.** Wat valt je op aan de opgeschoonde dataset? 



<span font-size=2.5em; style="color:red"><font size="5"><strong>Plenair bespreken en toelichting vervolg</strong></span>

<span font-size=2.5em;> Dus... </font></span><span font-size=2.5em; style="color:red"><font size="5"><strong>STOP </strong></span><span font-size=2.5em;> met verder klikken anders mis je de boot...</font></span>



    


**Opgave 3.2.** Voeg jezelf toe aan de dataset via onderstaande cel.

In [9]:
df_train_extended, df_test_extended = voeg_passagiers_toe(df_train_num, df_test_num)
display(df_test_extended.tail())

Wat is je naam?
Een leeg antwoord is niet bruikbaar.
Een leeg antwoord is niet bruikbaar.
Je hebt ingevuld: Luuk 

Wat is je geslacht (man, vrouw)?
Je hebt ingevuld: Man 

Wat is je leeftijd?
Je hebt ingevuld: 20 

Hoeveel kinderen neem je mee op reis?
Je hebt ingevuld: 0 

Hoeveel andere familieleden gaan mee op reis?
Je hebt ingevuld: 0 

In welk land ben je aan boord gegaan (Engeland, Frankrijk, Ierland)?
Je hebt ingevuld: Engeland 

Geef aan welke optie je voorkeur geniet voor de overige variabelen (a, b, c):
    a. 1e klasse ticket (prijs 749.99)
    b. 2e klasse ticket (prijs 99.99)
    c. 3e klasse ticket (prijs 7.50)

Je hebt ingevuld: A 

Wil je nog een passagier toevoegen (ja,nee)?
Je hebt ingevuld: Nee 



,,Geslacht,Leeftijd,Aantal_kinderen,Aantal_overige_familieleden,Totaal_aantal_familieleden,Opstapplaats,Ticket_klasse,Ticket_prijs
Passagier_Id,Naam,,,,,,,,
1306,"Oliva y Ocana, Dona. Fermina",1,39,0,0,0,1,1,108.900
1307,"Saether, Mr. Simon Sivertsen",0,38,0,0,0,0,3,7.250
1308,"Ware, Mr. Frederick",0,30,0,0,0,0,3,8.050
1309,"Peter, Master. Michael J",0,30,1,1,2,1,3,22.358
10000,Luuk,0,20,0,0,0,0,1,749.990


### 4. Modeling
Voer het onderstaande experiment uit met een veranderende parameter 'buren' (van 1 t/m 10).  
Kijk wat een verandering in het aantal buren met de nauwkeurigheidsscore doet.

In [10]:
df_experiment = experimenteer_met_aantal_buren(df=df_train_extended, ondergrens=1, bovengrens=10)
display(df_experiment)

,Aantal_buren,Nauwkeurigheidsscore
1,1,0.713
2,2,0.669
3,3,0.742
4,4,0.697
5,5,0.713
6,6,0.691
7,7,0.702
8,8,0.702
9,9,0.719
10,10,0.697


**Opgave 4.1.** Vraag voor gevorderde(n): Waarom zou een ‘even’ aantal buren niet aan te raden zijn?

### 5. Evaluation
Laten we hieronder bij de passagiers voorspellen of ze het overleefd hebben, waaronder ons zelf! 

In [11]:
df_voorspelling = voorspelling_genereren(X=df_test_extended)
display(df_voorspelling.tail())

,,Geslacht,Leeftijd,Aantal_kinderen,Aantal_overige_familieleden,Totaal_aantal_familieleden,Opstapplaats,Ticket_klasse,Ticket_prijs,Overleefd
Passagier_Id,Naam,,,,,,,,,
1306,"Oliva y Ocana, Dona. Fermina",1,39,0,0,0,1,1,108.900,1.000
1307,"Saether, Mr. Simon Sivertsen",0,38,0,0,0,0,3,7.250,0.000
1308,"Ware, Mr. Frederick",0,30,0,0,0,0,3,8.050,0.000
1309,"Peter, Master. Michael J",0,30,1,1,2,1,3,22.358,1.000
10000,Luuk,0,20,0,0,0,0,1,749.990,1.000


Mocht er in de kolom 'Overleefd' een '1' staan, dan heb je het overleefd! Mocht er een '0' staan, dan is het resultaat wellicht teleurstellend.   
Maar wees gerust, er zijn een aantal variabelen doorslaggevend en bij een goed doorlopen CRISP-DM proces heb je deze in beeld en kun je (vroegtijdig) ingrijpen.  

In onderstaande visualisatie kun je zien welke variabelen doorgeslaggevend waren. 

In [12]:
geef_belangrijkste_variabelen(df_train_num)

De variabelen welke de meeste invloed uitoefenen op het resultaat zijn:


**Opgave 5.1.** In welke mate komt de inschatting bij opgave 2.1 overeen met de visualisatie van variabelen die de meeste invloed uitoefenen?

### 6. Deployment
Deployment gaat te ver voor de introductie in Data Science. Tijd voor de wrap up. 